### Batch_Testing

In [3]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from datetime import datetime
import time
import socket
import struct
import copy
pd.set_option('mode.chained_assignment', None)
import warnings


# #######Define logdf to store logs for visualization
# Logdf = pd.DataFrame(columns=('Model', 'K1', 'K2', 'w1', 'w2', 'ROC_AUC', 'PRC_AUC', 'TimeCost'))

# mlist=[0,1,1,1,1,1,1,2,3,4,5]
# nlist=[1,0,1,2,3,4,5,1,1,1,1]
# for K_1 in range(1,5):
#     for K_2 in range(1,5):
#         for k in range(len(mlist)):
#             w_1 = mlist[k]
#             w_2 = nlist[k]


#############Loading raw data#############
dfinit = pd.read_csv("data/loginmysql-db_for_exp.csv")
dfinit.head()

#################Data feature processing#######################

dfinit['tih'] = pd.to_datetime(dfinit['tih'], format='%Y-%m-%d')
dfinit['day']=dfinit['tih'].dt.day
dfinit['hour']=dfinit['tih'].dt.hour

groups_sip_dip = dfinit.groupby(['sip','dip'])
groups_sip_dip_sum = pd.DataFrame(dfinit.groupby(['sip','dip'])['num'].sum())
groups_sip_dip_sqlsum = pd.DataFrame(dfinit.groupby(['sip','dip'])['sqlnum'].sum())
groups_sip_dip_dayuniq = pd.DataFrame(dfinit.groupby(['sip','dip'])['day'].nunique())

dfdict = {}
i = 0
for (k1,k2),groupi in groups_sip_dip:
    dfexpi_sip_dip = groupi
    dfexpi_sip_dip['numSum'] = groups_sip_dip_sum['num'][i]
    dfexpi_sip_dip['sqlnumSum'] = groups_sip_dip_sqlsum['sqlnum'][i]
    dfexpi_sip_dip['nday'] = groups_sip_dip_dayuniq['day'][i]
    dfdict[i] = dfexpi_sip_dip
    i+=1
dfdict_pd = pd.concat(list(dfdict.values()), ignore_index=True)
# dfdict_pd.head()

df = copy.deepcopy(dfdict_pd)
for i in range(df.shape[0]):
    df.loc[i,'sip'] = socket.ntohl(struct.unpack("I",socket.inet_aton(str(df.loc[i,'sip'])))[0])
    df.loc[i,'dip'] = socket.ntohl(struct.unpack("I",socket.inet_aton(str(df.loc[i,'dip'])))[0])

df['No.'] = ""
for i in range(df.shape[0]):
    df.loc[i,'No.']=i
# df.head()

df_only_login_init = df[df["dbtype"]=="0"]
df_with_db_init = df[df["dbtype"]=="db"]

df_only_login = copy.deepcopy(df_only_login_init)
df_with_db = copy.deepcopy(df_with_db_init)


#############Defining Parameters and Functions#############

# Calculate the distance of each data point to its cluster center
def getDistanceByPoint(data, model,oridata, div):
    distance = pd.Series()
    if div == True:
        for i in range(0,len(data)):
            Xa = np.array(data.loc[i])
            Xb = model.cluster_centers_[model.labels_[i]]
            distance.at[i] = np.linalg.norm(Xa-Xb)
            distance.at[i] = distance.at[i] / oridata.loc[i,"nday"] #######Divide by the frequency to get the anomaly score
    else:
        for i in range(0,len(data)):
            Xa = np.array(data.loc[i])
            Xb = model.cluster_centers_[model.labels_[i]]
            distance.at[i] = np.linalg.norm(Xa-Xb)

    return distance

min_max_scaler_01 = preprocessing.MinMaxScaler(feature_range=(0,1), copy=True)

dfabcorrect=['10.49.223.194','10.49.212.206','10.49.141.206','10.56.34.134','10.49.253.35',
             '10.49.179.13','10.56.148.80',
             '10.32.240.119','10.49.21.125','10.56.148.238']
dfabcorrect_set = set(dfabcorrect)


IPnumlist = pd.concat([dfinit["sip"], dfinit["dip"]], axis=0)
IPnumlist=IPnumlist.drop_duplicates()
IPnumlist.reset_index(drop=True, inplace=True)
IPnum= IPnumlist.shape[0]

def Metric(abIP,TP,FN):
    FP=abIP-TP
    TN=IPnum-abIP-FN
    R= TP/(TP+FN)
    TPR = TP/(TP+FN)
    FPR = FP/(TN+FP)
    P= TP/(TP+FP)
    return round(R, 4),round(TPR, 4),round(FPR, 4),round(P, 4)

Dslist = [df_only_login,df_with_db]
final_dslist = []

#Define K1 K2 w1 w2 ,model(k-means++ or random)
K_1=2
K_2=1
w_1=1
w_2=3
# model = "k-means++"
model = "random"


def AbDetect(K1,K2,w1,w2,model):
    for ds in Dslist:
        #########Group by < hour,sip,dip >###########
        mmfeature=[x for x in ds.columns if x not in ['tih','sip','dip', 'No.','dbtype','hour','day','nday','numSum','sqlnumSum']]
        kmfeature = pd.DataFrame((ds[mmfeature] != 0).any(axis=0))
        kmfeaturelist = kmfeature[kmfeature[0]==True].index.tolist()
        kmfeature_len = len(kmfeaturelist)

        dfexp = copy.deepcopy(ds)
        groups_hsd = dfexp.groupby(['hour','sip','dip'])

        holding = {}
        i = 0
        for (k1,k2,k3),group in groups_hsd:
            dfexpi = group
            ###reset_index
            dfexpi.reset_index(drop=True, inplace=True)
            ######Normalization
            for col in dfexpi[mmfeature]:
                dfexpi[[col]]=min_max_scaler_01.fit_transform(dfexpi[[col]])

            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                # Get the distance of each point to the cluster center
                if(dfexpi.shape[0] >= K1):
                    distance = getDistanceByPoint(dfexpi[kmfeaturelist], KMeans(n_clusters=K1,init=model).fit(dfexpi[kmfeaturelist]),dfexpi,False)
                else:
                    distance = getDistanceByPoint(dfexpi[kmfeaturelist], KMeans(n_clusters=1,init=model).fit(dfexpi[kmfeaturelist]),dfexpi,False)
            dfexpi['tsp_distance'] = distance
            holding[i] = dfexpi
            i+=1
        tsp_final = pd.concat(list(holding.values()), ignore_index=True)

        ### Group by < sip >
        dfexp_sip = copy.deepcopy(ds)
        column_uniques_sip = dfexp_sip['sip'].unique() 
        groups_sip = dfexp_sip.groupby('sip')
        kmfeature_sip=[x for x in dfexp_sip.columns if x in ['dip','numSum','sqlnumSum']]

        holding_sip = {}
        i = 0
        for column_sip in column_uniques_sip:
            dfexpi_sip = groups_sip.get_group(column_sip)
            dfexpi_sip.reset_index(drop=True, inplace=True)
            ######Normalization
            dfexpi_sip[['dip']] = min_max_scaler_01.fit_transform(dfexpi_sip[['dip']])
            dfexpi_sip[['numSum']] = min_max_scaler_01.fit_transform(dfexpi_sip[['numSum']])
            dfexpi_sip[['sqlnumSum']] = min_max_scaler_01.fit_transform(dfexpi_sip[['sqlnumSum']])

            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                #  Get the distance of each point to the cluster center
                if(dfexpi_sip.shape[0] >= K2):
                    distance_sip = getDistanceByPoint(dfexpi_sip[kmfeature_sip], KMeans(n_clusters=K2,init=model).fit(dfexpi_sip[kmfeature_sip]),dfexpi_sip,True)
                else:
                    distance_sip = getDistanceByPoint(dfexpi_sip[kmfeature_sip], KMeans(n_clusters=1,init=model).fit(dfexpi_sip[kmfeature_sip]),dfexpi_sip,True)

            dfexpi_sip['sip_distance'] = distance_sip
            holding_sip[i] = dfexpi_sip
            i+=1
        sipfinal = pd.concat(list(holding_sip.values()), ignore_index=True)


        ### Group by < dip >
        dfexp_dip = copy.deepcopy(ds)
        column_uniques_dip = dfexp_dip['dip'].unique()
        groups_dip = dfexp_dip.groupby('dip')
        kmfeature_dip=[x for x in dfexp_dip.columns if x in ['sip','numSum','sqlnumSum']]
        holding_dip = {}
        i = 0
        for column_dip in column_uniques_dip:
            dfexpi_dip = groups_dip.get_group(column_dip)
            dfexpi_dip.reset_index(drop=True, inplace=True)
            ######Normalization
            dfexpi_dip[['sip']] = min_max_scaler_01.fit_transform(dfexpi_dip[['sip']])
            dfexpi_dip[['numSum']] = min_max_scaler_01.fit_transform(dfexpi_dip[['numSum']])
            dfexpi_dip[['sqlnumSum']] = min_max_scaler_01.fit_transform(dfexpi_dip[['sqlnumSum']])

            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                #  Get the distance of each point to the cluster center
                if(dfexpi_dip.shape[0] >= K2):
                    distance_dip = getDistanceByPoint(dfexpi_dip[kmfeature_dip], KMeans(n_clusters=K2,init=model).fit(dfexpi_dip[kmfeature_dip]),dfexpi_dip,True)
                else:
                    distance_dip = getDistanceByPoint(dfexpi_dip[kmfeature_dip], KMeans(n_clusters=1,init=model).fit(dfexpi_dip[kmfeature_dip]),dfexpi_dip,True)

            dfexpi_dip['dip_distance'] = distance_dip
            holding_dip[i] = dfexpi_dip
            i+=1
        dipfinal = pd.concat(list(holding_dip.values()), ignore_index=True)
        
        ### Integrate ds
        #### ！Important
        tsp_final.sort_values(by="No." , inplace=True, ascending=True)
        sipfinal.sort_values(by="No." , inplace=True, ascending=True)
        dipfinal.sort_values(by="No." , inplace=True, ascending=True)
        #### ！Important reset_index
        tsp_final.reset_index(drop=True, inplace=True)
        sipfinal.reset_index(drop=True, inplace=True)
        dipfinal.reset_index(drop=True, inplace=True)
        
        tsp_final['sip_distance'] = sipfinal['sip_distance']
        tsp_final['dip_distance'] = dipfinal['dip_distance']
        wadjust = (kmfeature_len**0.5/(len(kmfeature_dip)**0.5/1))
#         wadjust = 1
        
        #######Calculate the final score
        tsp_final['final_distance'] = w1*tsp_final['tsp_distance'] + wadjust*w2*tsp_final['sip_distance'] + wadjust*w2*tsp_final['dip_distance']###adjust the weights

        ds.reset_index(drop=True, inplace=True)
        ds['final_distance'] = tsp_final['final_distance']
        final_ds = copy.deepcopy(ds)

        for i in range(final_ds.shape[0]):
            final_ds.loc[i,'sip'] = socket.inet_ntoa(struct.pack('I',socket.htonl(int(final_ds.loc[i,'sip']))))
            final_ds.loc[i,'dip'] = socket.inet_ntoa(struct.pack('I',socket.htonl(int(final_ds.loc[i,'dip']))))
            
        final_ds.sort_values(by="No.",inplace=True, ascending=True)
        
        final_dslist.append(final_ds)
        
    final = reduce(lambda left,right: pd.merge(left,right,how='outer'),final_dslist)
    finalsort_ = final.sort_values(["final_distance"],ascending=False)
    finalsort_.reset_index(drop=True, inplace=True)
    return finalsort_

#######Define logdf to store logs for visualization
Logdf = pd.DataFrame(columns=('Model','K1','K2','w1','w2','ROC_AUC','PRC_AUC','TimeCost'))

time_start = time.time()
#########Detection########
finalsort = AbDetect(K_1,K_2,w_1,w_2,model)
time_end = time.time()
cost = round((time_end - time_start),4)

#######Define Lists to keep TPR FPR R P
TPRlist = []
FPRlist = []
Plist = []
Rlist = []

num=1
while num < finalsort.shape[0]+1:
    ab_finalsort= finalsort[0:num]
    num += 1
    dfabip=pd.concat([ab_finalsort["sip"], ab_finalsort["dip"]], axis=0)
    dfabip=dfabip.drop_duplicates()
    dfabip.reset_index(drop=True, inplace=True)

    dfabip.tolist()
    dfabip_set = set(dfabip)
    bingo=(dfabip_set)&(dfabcorrect_set)
    abIPi=len(dfabip)
    TPi=len(bingo)
    FNi=len(dfabcorrect)-TPi

    R,TPR,FPR,P = Metric(abIPi,TPi,FNi)
    TPRlist.append(TPR)
    FPRlist.append(FPR)
    Plist.append(P)
    Rlist.append(R)

ROC = pd.DataFrame(FPRlist, columns=['FPR'])
ROC = pd.concat([ROC, pd.DataFrame(TPRlist,columns=['TPR'])],axis=1)
PR = pd.DataFrame(Rlist, columns=['R'])
PR = pd.concat([PR, pd.DataFrame(Plist,columns=['P'])],axis=1)

####Importance drop_duplicates
ROC=ROC.drop_duplicates()
PR=PR.drop_duplicates()

####Importance sort_values
ROC.sort_values(by=['FPR','TPR'],inplace=True,ascending=True)
PR.sort_values(by=['R','P'],inplace=True,ascending=[True,False])
ROC.reset_index(drop=True, inplace=True)
PR.reset_index(drop=True, inplace=True)


#####Importance Add the last line For ROC
ROC.loc[ROC.shape[0]] = [1,ROC.loc[ROC.shape[0]-1,'TPR']]
#####Importance Add the first line For PR
PR.loc[-1] = [0,PR.loc[0,'P']]
PR.index = PR.index + 1
PR.sort_index(inplace=True)


#####AUC
ROC_AUC = np.trapz(list(ROC['TPR']),list(ROC['FPR'])).round(4)
PRC_AUC = np.trapz(list(PR['P']),list(PR['R'])).round(4)

######Save Logs
# Path_ROC = "Results/Batch_Testing/ROC_Model%s_K%d%d_w%d%d.csv"%(model,i,j,m,n)
# ROC.to_csv(Path_ROC,index=0)
# Path_PRC = "Results/Batch_Testing/PRC_Model%s_K%d%d_w%d%d.csv"%(model,K_1,K_2,w_1,w_2)
# PR.to_csv(Path_PRC,index=0)

Logdf = Logdf.append({'Model':model,'K1':K_1,'K2':K_2,'w1':w_1,'w2':w_2,'ROC_AUC':ROC_AUC,'PRC_AUC':PRC_AUC,'TimeCost':cost}, ignore_index=True)
# Logdf.to_csv("Results/Batch_Testing/All_Logs.csv",index=0)
print(Logdf)

    Model K1 K2 w1 w2  ROC_AUC  PRC_AUC  TimeCost
0  random  2  1  1  3    0.986   0.9665   27.9673


In [4]:
# Logdf.to_csv("Results/Batch_Testing/All_Logs.csv",index=0)

### Extraction for DFADR

In [5]:
finalsort = AbDetect(2,1,1,3,model)

##########Take R=1 and P is the highest
TPRlist = []
FPRlist = []
Plist = []
Rlist = []
numlist = []

num = 1
while num < finalsort.shape[0]+1:


    ab_finalsort= finalsort[0:num]
    num += 1
    
    dfabip=pd.concat([ab_finalsort["sip"], ab_finalsort["dip"]], axis=0)
    dfabip=dfabip.drop_duplicates()
    dfabip.reset_index(drop=True, inplace=True)

    # print(dfabip)

    dfabip.tolist()
    dfabip_set = set(dfabip)
    
    bingo=(dfabip_set)&(dfabcorrect_set)

    abIPi=len(dfabip)
    TPi=len(bingo)
    FNi=len(dfabcorrect)-TPi
    
    R,TPR,FPR,P = Metric(abIPi,TPi,FNi)
    
    #####Only Keep R==1
    if R == 1:
        numlist.append(num-1)
        Plist.append(P)
        Rlist.append(R)
    
PRKeep = pd.DataFrame(Rlist, columns=['R'])
PRKeep = pd.concat([PRKeep, pd.DataFrame(Plist,columns=['P'])],axis=1)
PRKeep = pd.concat([PRKeep, pd.DataFrame(numlist,columns=['num'])],axis=1)


PRKeep.sort_values(by=['R','P'],inplace=True,ascending=True)
PRKeep.drop_duplicates(subset=['R'],keep='last', inplace=True)
PRKeep.reset_index(drop=True, inplace=True)
PRKeep

# ####Importance drop_duplicates
# ROC=ROC.drop_duplicates()
# PR=PR.drop_duplicates()

# ####Importance sort_values
# ROC.sort_values(by=['FPR','TPR'],inplace=True,ascending=True)
# PR.sort_values(by=['R','P'],inplace=True,ascending=True)
        
# ROC.reset_index(drop=True, inplace=True)
# PR.reset_index(drop=True, inplace=True)

,R,P,num
0,1.0,0.8333,46


In [7]:
PRKeep['num'][0]

46

In [8]:
ab_final_output= finalsort[0:PRKeep['num'][0]]
ab_final_output.head()

,tih,sip,dip,num,difsport,difpw,difinfo,difuser,sqlnum,difsqlinfo,dbtype,day,hour,numSum,sqlnumSum,nday,No.,final_distance
0,2019-06-11 19:00:00,10.49.141.206,10.49.253.35,1,1,1,1,1,0,0,0,11,19,1,0,1,220,7.127453
1,2019-06-11 21:00:00,10.49.223.194,10.49.253.35,13,13,13,1,1,17,3,db,11,21,13,17,1,897,6.778072
2,2019-06-11 19:00:00,10.49.141.206,10.49.253.35,1,1,1,1,1,0,0,0,11,19,1,0,1,220,6.506444
3,2019-06-11 21:00:00,10.49.223.194,10.49.253.35,13,13,13,1,1,17,3,db,11,21,13,17,1,897,6.340306
4,2019-06-06 19:00:00,10.56.34.134,10.49.253.35,1,1,1,1,1,4,4,db,6,19,8,32,2,1279,4.153079


In [37]:
ab_final_output.to_csv("Results/WebBackend_for_DFADR.csv",index=0)